In [22]:
## import useful packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
pd.options.display.max_rows = 50

In [23]:
# read from the data csv
df = pd.read_csv('electrical_stability.csv') 
df.rename(columns={"stabf" : "stability"}, inplace=True)
df # whenever stab >0, stabf = unstable and stab <0, stabf = stable

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stability
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2.930406,9.487627,2.376523,6.187797,3.343416,-0.658054,-1.449106,-1.236256,0.601709,0.779642,0.813512,0.608385,0.023892,unstable
9996,3.392299,1.274827,2.954947,6.894759,4.349512,-1.663661,-0.952437,-1.733414,0.502079,0.567242,0.285880,0.366120,-0.025803,stable
9997,2.364034,2.842030,8.776391,1.008906,4.299976,-1.380719,-0.943884,-1.975373,0.487838,0.986505,0.149286,0.145984,-0.031810,stable
9998,9.631511,3.994398,2.757071,7.821347,2.514755,-0.966330,-0.649915,-0.898510,0.365246,0.587558,0.889118,0.818391,0.037789,unstable


In [ ]:
df.info()

In [ ]:
# Removing outliers from the dataset
from scipy.stats import iqr
def remove_outliers(df):
    for c in df.columns:
            pct_75 = np.percentile(df[c], 75)
            pct_25 = np.percentile(df[c], 25)
            upper_bound = pct_75 + 1.5*iqr(df[c])
            lower_bound = pct_25 - 1.5*iqr(df[c])
            condition = (df[c] < upper_bound) & (df[c] > lower_bound)
            df[c] = df[c][condition]  # Filter out the outliers
    return df

In [ ]:
# Dummy values for the 'stabf' column: stabf has two outputs given by 1. stable 2. unstable
#dummy_columns = df.select_dtypes(include='object').columns # columns to dummify: stabf is the only categorical value
#df_dummies = pd.get_dummies(df, prefix=dummy_columns)
#df_dummies

In [ ]:
# correlation heat map: the first insight into the data
corr = df.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True # trick to filter out the upper-right triangle, which is redundant due to symmetry
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(16, 14))
    ax = sns.heatmap(corr, mask=mask,cmap='coolwarm', vmin=-1,vmax=1,annot=True, square=True)

In [ ]:
# Just for showing a plot between average tau and stability
df["tau_avg"] = pd.DataFrame(0.25*(df["tau1"] + df["tau2"] + df["tau3"] + df["tau4"]))
graph = sns.boxplot(y=df['tau_avg'], x=df["stability"])
graph.set(ylabel="Average time constant", xlabel = "Stability")
#graph.axvline(4.5, c='g',  linestyle="dashed")
#graph.axvline(6, c='r',  linestyle="dashed")

In [ ]:
df.iloc[0:10].plot(y=['p1', 'p2', 'p3', 'p4'])

In [ ]:
## Prediction and modeling

# test-train split
RAND_STATE = 34 # for reproducible shuffling
TT_RATIO = 0.3 # test/train

X = df.drop('stab', axis=1).drop('stability', axis=1).drop('tau_avg', axis=1) # all the variables, except 'stab' and 'stabf' and tau_avg
y = df.stab # 'stab' is our numerical output variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TT_RATIO, random_state=RAND_STATE) # test-train split
X_train = pd.DataFrame(X_train) 
X_test = pd.DataFrame(X_test)
X_train

In [ ]:
# Scaling
from sklearn.preprocessing import PowerTransformer, StandardScaler, MinMaxScaler
scaler = PowerTransformer()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [ ]:
# Model and training
model=LinearRegression()    # model
model.fit(X_train_scaled, y_train)   # model train
model.coef_

In [ ]:
# Feature importance
features_importances = pd.DataFrame(data={
    'Attribute': X_train.columns,
    'Importance': abs(model.coef_.reshape(len(X_train.columns),))
})
features_importances = features_importances.sort_values(by='Importance', ascending=False)
features_importances

In [ ]:
# Feature importance modeling
plt.bar(x=features_importances['Attribute'].iloc[:], height=features_importances['Importance'].iloc[:], color='#087E8B')
plt.title('Importance of each feature to affect sability', size=12)
plt.xticks(rotation='vertical')
plt.xlabel("Input features")
plt.ylabel("Importance on stability")
plt.show()

In [ ]:
# Prediction
y_test_pred = pd.DataFrame(model.predict(X_test_scaled),columns = ['stab'] )      # model prediction
y_train_pred =  pd.DataFrame(model.predict(X_train_scaled),columns = ['stab'])

In [ ]:
# Prediction versus actual plot

fig, ax = plt.subplots(1,2,figsize=(14,4))
ax[0].scatter(y_test, y_test_pred, c='crimson')
ax[0].set_xlabel('Actual values', fontsize=12)
ax[0].set_ylabel('Predicted values', fontsize=12)

z = pd.DataFrame(y_test)
e = y_test_pred['stab'] - z['stab']

ax[1].scatter(e.index, e, c='green')
ax[1].set_xlabel('Data points', fontsize=12)
ax[1].set_ylabel('Residual (errors)', fontsize=12)
ax[1].axhline(0, c='b')

In [ ]:
print(mse(y_test,y_test_pred))



In [ ]:
R2_test=r2_score(y_test,y_test_pred) # R2 score between the predicted Y test and the actual Y test
R2_train=r2_score(y_train,y_train_pred) # R2 score between the predicted Y train and the actual Y train
print(R2_test) 
print(R2_train)


In [ ]:
# Model Evaluation: Adjusted R2
Adj_R2_test= 1 - (1-R2_test)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1) # len(y_test) = number of samples for the test set, X_test.shape[1] = the number of independent variables for the test set
Adj_R2_train= 1 - (1-R2_train)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
print(Adj_R2_test) 
print(Adj_R2_train)